<a href="https://colab.research.google.com/github/avisekksarma/Fraud-detection/blob/main/fe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive' , force_remount=True)

Mounted at /content/gdrive


In [ ]:
import os
import pandas as pd
from sklearn import preprocessing
import numpy as np
import lightgbm as lgb


In [ ]:
os.chdir('//content/gdrive/My Drive/Fraud_detection/')

print('Load Data')
train_df = pd.read_pickle('train_transaction.pkl')
test_df = pd.read_pickle('test_transaction.pkl')
test_df['isFraud'] = 0

train_identity = pd.read_pickle('train_identity.pkl')
test_identity = pd.read_pickle('test_identity.pkl')

Load Data


FileNotFoundError: ignored

card4,card6 and ProductCD are frequency encoded (prior)

In [ ]:
# train_df[['card1','card6','ProductCD']].head()

M4 is frequency encoded . M1-M9 values describe card match features

In [ ]:
# train_df.loc[:, 'M1':'M9'].head()

In [ ]:
# Here I shall do Feature Engineering and implement a baseline model

In many practical data science activities, the data set will contain categorical variables. These variables are typically stored as text values”. Since machine learning is based on mathematical equations, it would cause a problem when we keep categorical variables as is. Many algorithms support categorical values without further manipulation, but in those cases, it’s still a topic of discussion on whether to encode the variables or not. The algorithms that do not support categorical values, in that case, are left with encoding methodologies.

#### **FE and Model**

Crucial steps

1.Make ground baseline with no fe <br>
2.Make a small FE and see I you can understand data you have <br>
3.Find good CV strategy <br>
4.Feature selection <br>
5.Make deeper FE <br>
6.Tune Model (crude tuning) <br>
7.Try other Models (never forget about NN) <br>
8.Try Blending/Stackin/Ensembling <br>
9.Final tuning <br>

##### **1.BaseLine Model**

In [ ]:
# 1. Baseline Model . It doesn't really matter what model to use her. Make ground baseline. No feature engineering. No tuning.

train = train_df.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_df.merge(test_identity, how='left', left_index=True, right_index=True)

In [ ]:
# print(train.shape)
# print(test.shape)

target = train['isFraud'].copy()
X_train = train.drop('isFraud', axis=1)
X_train.drop('TransactionDT', axis=1, inplace=True)
X_test = test.drop('TransactionDT', axis=1)

# Drop target, fill in NaNs

X_test = test.copy()
X_test.columns = X_test.columns.str.replace('-','_')

# If you give np.nan to LGBM, then at each tree node split,
# it will split the non-NAN values and then send all the NANs to either the left child or right child depending on what’s best.
# Therefore NANs get special treatment at every node and can become overfit.
# By simply converting all NAN to a negative number lower than all non-NAN values (such as - 999),

X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

In [ ]:
del train,test

In [ ]:
import gc

gc.collect()

In [ ]:
# X_train.head()

In [ ]:
# X_test.head()

In [ ]:
columns = []
for f in X_train.columns:
  if X_train[f].dtype == 'object':
    columns.append(f)

print(columns)

In [ ]:
#Frequency Encoding

def frequency_encoding(train_df, test_df, columns,self_encoding=True):
    for col in columns:
        temp_df = pd.concat([train_df[[col]], test_df[[col]]])
        fq_encode = temp_df[col].value_counts(dropna=False).to_dict()

        if self_encoding:
            train_df[col] = train_df[col].map(fq_encode)
            test_df[col]  = test_df[col].map(fq_encode)
        else:
            train_df[col+'_fq_enc'] = train_df[col].map(fq_encode)
            test_df[col+'_fq_enc']  = test_df[col].map(fq_encode)
    return train_df, test_df

X_train,X_test = frequency_encoding(X_train,X_test , columns)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
# X_train = X_train.drop(['TransactionID_x'],axis=1)
# X_train.head()

In [ ]:
# X_test = X_test.drop(['TransactionID_x'],axis=1)
# X_test.head()

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [ ]:
SEED = 10

params = {'num_leaves': 256,
          'min_child_samples': 79,
          'objective': 'binary',
          'max_depth': 13,
          'learning_rate': 0.03,
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.9,
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.9,
          #'categorical_feature': cat_cols
         }

In [ ]:
splits = 5
folds = KFold(n_splits = splits)
oof = np.zeros(len(X_train))
predictions = np.zeros(len(X_test))

In [ ]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train.values, target.values)):
    print("Fold {}".format(fold_))
    train_df, y_train_df = X_train.iloc[trn_idx], target.iloc[trn_idx]
    valid_df, y_valid_df = X_train.iloc[val_idx], target.iloc[val_idx]

    trn_data = lgb.Dataset(train_df, label=y_train_df)
    val_data = lgb.Dataset(valid_df, label=y_valid_df)

    clf = lgb.train(params,
                    trn_data,
                    10000,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=500,
                    early_stopping_rounds=500)

    pred = clf.predict(valid_df)
    oof[val_idx] = pred
    print( "  auc = ", roc_auc_score(y_valid_df, pred) )
    predictions += clf.predict(X_test) / splits

##### **2.Drop Features**

In [ ]:
#TODO :
# 1. view Nans and drop those with 70% nans or else fill nans
# 2. drop redundant features check corelation and remove
# 3. drop less important features








Aggregation ? <br>
D Column <br>
Clip Transaction Amount <br>



##### **3.Normalize Data**

##### **4.Outlier Removal**

##### **5.PCA**

##### **5.CV strategy**